In [1]:
import json

In [15]:
# file = 'cse.json'
file = "/Users/adebayobraimah/Downloads/drive-download-20240320T171548Z-001/sample_json_document.json"

In [20]:
def write_ergo(data, file, parent_oid=None):
    # DOC: Recursively write data to an Ergo formatted file.
    if parent_oid is None:
        parent_oid = "root"
        file.write(f"?- \\@[add_json_{parent_oid}(\\##{parent_oid})]@\\json.\n")

    if isinstance(data, dict):
        for key, value in data.items():
            if isinstance(value, dict):
                new_oid = f"\\##{key}"
                file.write(f"\\##{parent_oid}['{key}' -> {new_oid}].\n")
                write_ergo(value, file, new_oid)
            elif isinstance(value, list):
                list_items = ", ".join(
                    [
                        f"\\##{key}_{i}" if isinstance(v, dict) else str(v)
                        for i, v in enumerate(value)
                    ]
                )
                file.write(f"\\##{parent_oid}['{key}' -> [{list_items}]].\n")
                for i, v in enumerate(value):
                    if isinstance(v, dict):
                        write_ergo(v, file, f"\\##{key}_{i}")
            else:
                if isinstance(value, str):
                    value = value.replace("\\", "\\\\")
                    value = f"'{value}'"
                elif isinstance(value, bool):
                    value = str(value).lower()
                file.write(f"\\##{parent_oid}['{key}' -> {value}].\n")
    elif isinstance(data, list):
        # This case is for lists if you want to handle top-level arrays.
        for i, item in enumerate(data):
            new_oid = f"\\##{parent_oid}_{i}"
            file.write(f"\\##{parent_oid}[{i} -> {new_oid}].\n")
            write_ergo(item, file, new_oid)

In [21]:
def convert_json_to_ergo(json_filepath, ergo_filepath):
    with open(json_filepath, "r", encoding="utf-8") as f:
        data = json.load(f)

    with open(ergo_filepath, "w", encoding="utf-8") as f:
        f.write(":- new_global_oid_scope.\n")  # Initialize the global OID scope.
        write_ergo(data, f)
        f.write(
            "?- new_global_oid_scope.\n"
        )  # Reset global OID scope after completion.

In [22]:
convert_json_to_ergo(file, "output.ergo")